1. 26_Data.csv 데이터를 df_product로, 25_Data.csv를 df_distribution으로 선언하고, df_product 데이터와 df_distribution데이터를 df_distribution데이터를 기준으로 병합해 df1으로 선언하시오.을 생성하시오.

In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats

In [6]:
df_product = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\26_Data.csv')
print(df_product.shape)
df_product.info()

(3611, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3611 entries, 0 to 3610
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품코드    3611 non-null   object
 1   제품명     3611 non-null   object
 2   물품가격    3611 non-null   int64 
 3   대분류     3611 non-null   object
 4   소분류     3611 non-null   object
 5   제조사     2924 non-null   object
dtypes: int64(1), object(5)
memory usage: 169.4+ KB


In [7]:
df_distribution = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\25_Data.csv')
print(df_distribution.shape)
df_distribution.info()

(731761, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731761 entries, 0 to 731760
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  731761 non-null  int64  
 1   공급ID        731761 non-null  object 
 2   공급일자        731761 non-null  object 
 3   유통Hub       731761 non-null  object 
 4   판매금액        731761 non-null  int64  
 5   수량          731761 non-null  float64
 6   상품코드        731761 non-null  object 
 7   제품명         731761 non-null  object 
 8   Div         731761 non-null  object 
 9   본부          731761 non-null  object 
 10  채널          731761 non-null  object 
 11  지역          731761 non-null  object 
 12  영업대표        731761 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 72.6+ MB


In [8]:
df1 = pd.merge(df_distribution, df_product, on=['상품코드','제품명'], how='inner' )

In [9]:
print(df1.shape)
df1.info()

(731761, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731761 entries, 0 to 731760
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  731761 non-null  int64  
 1   공급ID        731761 non-null  object 
 2   공급일자        731761 non-null  object 
 3   유통Hub       731761 non-null  object 
 4   판매금액        731761 non-null  int64  
 5   수량          731761 non-null  float64
 6   상품코드        731761 non-null  object 
 7   제품명         731761 non-null  object 
 8   Div         731761 non-null  object 
 9   본부          731761 non-null  object 
 10  채널          731761 non-null  object 
 11  지역          731761 non-null  object 
 12  영업대표        731761 non-null  object 
 13  물품가격        731761 non-null  int64  
 14  대분류         731761 non-null  object 
 15  소분류         731761 non-null  object 
 16  제조사         606410 non-null  object 
dtypes: float64(1), int64(3), object(13)
memory usage: 94.9+ MB


2. 병합된 데이터를 기준으로 분석합니다. 가장 많이 유통된 상위 3개 상품 소분류에 대해, 상품 판매 금액의 유의미한 차이가 있는지 확인하시오.

In [32]:
pv1 = df1.pivot_table(index='소분류' , values='판매금액', aggfunc='sum').sort_values(by='판매금액', ascending=False).head(3)

In [59]:
cond1 = df1['소분류'] == pv1.index[0]
cond2 = df1['소분류'] == pv1.index[1]
cond3 = df1['소분류'] == pv1.index[2]

# 정규분포 따르는지. -> pvalue < 0.5 => 귀무가설 기각 => 정규분포XX
print(stasts.normaltest(df1.loc[cond1]['판매금액']))
print(stasts.normaltest(df1.loc[cond2]['판매금액']))
print(stasts.normaltest(df1.loc[cond3]['판매금액']))

NormaltestResult(statistic=259444.81532670674, pvalue=0.0)
NormaltestResult(statistic=12905.831110302788, pvalue=0.0)
NormaltestResult(statistic=69532.48459077894, pvalue=0.0)


In [40]:
df2 = df1.loc[cond1 | cond2 | cond3]

In [42]:
df2.shape

(228712, 17)

In [47]:
groups = df2.groupby('소분류')['판매금액'].apply(list)
stats.kruskal(*groups)
#  pvalue < 0.5 => 귀무가설 기각 => 대표값의 유의미한 차이가 있다.

KruskalResult(statistic=20509.731666588534, pvalue=0.0)

3. 가장 많이 유통되고 있는 제품에 대해 유통허브에 따라 유통수량의 유의미한 차이가 있는지 확인하시오.

In [52]:
p2 = df1.pivot_table(index='소분류', values='수량', aggfunc='sum').sort_values(by='수량', ascending=False)
p2

,수량
소분류,
바디케어세트,205126.31
치약/칫솔세트,44009.80
유아/아동용칫솔,35161.10
샴푸,34932.60
바디워시,28587.74
...,...
부분세탁제,18.00
남녀공용향수,15.00
태닝/애프터선케어,6.00


In [56]:
df3 = df1.loc[df1['소분류'] == '바디케어세트']
df3['유통Hub'].unique()

array(['경상', '서울/수도권', '충청/전라', '강원'], dtype=object)

In [60]:
cond_list = ['경상', '서울/수도권', '충청/전라', '강원']
for i in cond_list :
    print(stasts.normaltest(df3.loc[df3['유통Hub'] == i]['수량']))

NormaltestResult(statistic=48409.59379941935, pvalue=0.0)
NormaltestResult(statistic=65340.67854002927, pvalue=0.0)
NormaltestResult(statistic=62558.60568022526, pvalue=0.0)
NormaltestResult(statistic=65305.254565059266, pvalue=0.0)


In [61]:
groups = df3.groupby('유통Hub')['수량'].apply(list)
stats.kruskal(*groups)
#  pvalue < 0.5 => 귀무가설 기각 => 대표값의 유의미한 차이가 있다.

KruskalResult(statistic=54.69492178878355, pvalue=7.976188176531246e-12)

4. 유통허브가 각 유통채널에 상품을 적절하게 유통하고 있는지 확인하고자 한다. 유통허브 별로 유통 채널의 비율이 달라지는지 확인하시오.

In [63]:
df1['채널'].unique()

array(['현대Hmall', 'CJmall', '위메프', 'LotteMall', 'Auction', 'Naver쇼핑',
       'GS', 'TMON', 'Interpark', '11번가', '신세계Mall', 'AKMall', 'Coupang',
       'GMarket', '인터파크'], dtype=object)

In [64]:
df1['count'] = 1

In [77]:
p3 = df1.pivot_table(index=['유통Hub','채널'] , values='count', aggfunc='sum')
p3

count
유통Hub  채널              
강원     11번가        1579
       AKMall      1640
       Auction     3124
       CJmall     11037
       Coupang     8121
       GMarket     1530
       GS         11270
       Interpark  11240
       LotteMall   9468
       Naver쇼핑    23689
       TMON        3253
       신세계Mall     1554
       위메프         9518
       인터파크        1570
       현대Hmall    33330
경상     11번가        1694
       AKMall      1713
       Auction     3349
       CJmall     11492
       Coupang     8255
       GMarket     1608
       GS         11611
       Interpark  11570
       LotteMall   9898
       Naver쇼핑    24734
       TMON        3328
       신세계Mall     1668
       위메프         9904
       인터파크        1698
       현대Hmall    35033
서울/수도권 11번가        3668
       AKMall      3697
       Auction     7342
       CJmall     25450
       Coupang    18049
       GMarket     3670
       GS         25648
       Interpark  25579
       LotteMall  21950
       Naver쇼핑    54608
       TMON        7403
       신세계Mall     3722
       위메프        21880
       인터파크        3776
       현대Hmall    77234
충청/전라  11번가        1852
       AKMall      1842
       Auction     3914
       CJmall     13523
       Coupang     9652
       GMarket     1939
       GS         13281
       Interpark  13234
       LotteMall  11428
       Naver쇼핑    28862
       TMON        3791
       신세계Mall     1947
       위메프        11420
       인터파크        1900
       현대Hmall    40022

In [76]:
p3.groupby('유통Hub')['count'].sum()

유통Hub
강원        131923
경상        137555
서울/수도권    303676
충청/전라     158607
Name: count, dtype: int64

5. 22년도에 각 영업대표가 각 채널로 유통하는 유통판매금액의 합을 계산해, 22년 영업실적이 가장 높은 영업 대표를 확인하시오.
6. 데이터가 수집된 기간 내, 유통량이 온라인 소비자들의 수요와 관련이 있는지 확인하고자 한다. 데이터에 가장 많이 유통된 제품의 소분류를 확인하여, 아래 사이트를 활용해 해당 소분류를 소비자들이 검색했을 때, 검색량과 유통수량의 상관성이 있는지 분석하시오.
    - https://datalab.naver.com/
    - 성별 / 연령대 / 범위에 따라 각각 계산하시오.
7. 앞서 수집한 검색엔진 검색량 데이터를 X로, 유통수량을 Y로 머신러닝 알고리즘을 활용해, 특정 연령/성별/범위의 검색량에 대한 유통수량을 예측하는 모델을 생성하시오.